In [0]:
# Import required libraries
from pyspark.sql import functions as F

# Load november raw data
november_df = spark.read.csv("/Volumes/workspace/advecom/advecom_data/2019-Nov.csv", header=True, inferSchema=True)

In [0]:
# Create user feature dataframe, with necessary features only 
user_features_df = november_df.filter(F.col("price") > 0) \
    .filter("user_id IS NOT NULL") \
    .dropDuplicates(["user_id"]) \
    .groupBy("user_id") \
    .agg(
        F.sum(
            F.when(F.col("event_type")=="view",1)
            .otherwise(0)
        ).alias("total_views"),
        F.sum(
            F.when(F.col("event_type")=="cart",1)
            .otherwise(0)
        ).alias("total_cart"),
        F.sum(
            F.when(F.col("event_type")=="purchase",1)
            .otherwise(0)
        ).alias("total_purchases"),
        F.avg("price").alias("avg_price"),
        F.count("user_id").alias("total_events"),
        F.sum("price").alias("total_spent")
    )
    
# Check user features
display(user_features_df.limit(5))

user_id,total_views,total_cart,total_purchases,avg_price,total_events,total_spent
519209941,1,0,0,190.74,1,190.74
516612689,1,0,0,596.67,1,596.67
565525744,1,0,0,83.53,1,83.53
516625897,1,0,0,399.24,1,399.24
541547399,1,0,0,6.94,1,6.94


In [0]:
# Create target label dataframe from user features
labeled_df = user_features_df.withColumn(
    "label",
    F.when(F.col("total_purchases")>0,1)
    .otherwise(0)
    )

# Check labeled dataframe
display(labeled_df)

user_id,total_views,total_cart,total_purchases,avg_price,total_events,total_spent,label
519209941,1,0,0,190.74,1,190.74,0
516612689,1,0,0,596.67,1,596.67,0
565525744,1,0,0,83.53,1,83.53,0
516625897,1,0,0,399.24,1,399.24,0
541547399,1,0,0,6.94,1,6.94,0
565112441,1,0,0,124.82,1,124.82,0
522813774,1,0,0,193.02,1,193.02,0
517435676,1,0,0,1013.86,1,1013.86,0
563760136,1,0,0,67.9,1,67.9,0
532381974,1,0,0,476.17,1,476.17,0


In [0]:
# Check distribution of labels
labeled_df.groupBy("label").count().show()

+-----+-------+
|label|  count|
+-----+-------+
|    1|    654|
|    0|3694453|
+-----+-------+



In [0]:
# Create two dataframes for purchase and non-purchase, and then combine them with a smaller fraction of non purchase. As a result the model will be trained on a balanced dataset.
purchase_df = labeled_df.filter("label=1")

non_purchase_df = labeled_df.filter("label=0") \
.sample(fraction=0.0004)

balanced_df = purchase_df.union(non_purchase_df)

balanced_df.groupBy("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1|  654|
|    0| 1479|
+-----+-----+



In [0]:
# Create feature dataframe, only keeping the required columns
feature_df = balanced_df.select(
    "total_views",
    "total_cart",
    "total_spent",
    "total_events",
    "avg_price",
    "label"
)

In [0]:
# Import required libraries
from pyspark.ml.feature import VectorAssembler

# Create features vector for ML
assembler = VectorAssembler(
    inputCols=[
        "total_views",
        "total_cart",
        "total_spent",
        "total_events",
        "avg_price"
    ],
    outputCol="features"
)

# Create model dataframe with features and label
model_df = assembler.transform(feature_df).select("features","label")

display(model_df)

features,label
"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.0"",""0.0"",""98.33"",""1.0"",""98.33""]}",1
"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.0"",""0.0"",""360.09"",""1.0"",""360.09""]}",1
"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.0"",""0.0"",""128.42"",""1.0"",""128.42""]}",1
"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.0"",""0.0"",""1000.77"",""1.0"",""1000.77""]}",1
"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.0"",""0.0"",""99.1"",""1.0"",""99.1""]}",1
"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.0"",""0.0"",""174.78"",""1.0"",""174.78""]}",1
"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.0"",""0.0"",""31.38"",""1.0"",""31.38""]}",1
"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.0"",""0.0"",""15.42"",""1.0"",""15.42""]}",1
"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.0"",""0.0"",""566.3"",""1.0"",""566.3""]}",1
"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.0"",""0.0"",""28.29"",""1.0"",""28.29""]}",1


In [0]:
# Split data into training and test
train_df, test_df = model_df.randomSplit([0.8,0.2], seed=6)

# Check label distribution across train and test
train_df.groupBy("label").count().show()
test_df.groupBy("label").count().show() 

+-----+-----+
|label|count|
+-----+-----+
|    1|  524|
|    0| 1194|
+-----+-----+

+-----+-----+
|label|count|
+-----+-----+
|    1|  130|
|    0|  285|
+-----+-----+



In [0]:
# Import required libraries
from pyspark.ml.classification import LogisticRegression

# Initialize logistic regression model
lr = LogisticRegression(
    featuresCol="features",
    labelCol="label"

)

# Fit model on training data
lr_model = lr.fit(train_df)

# Make predictions on test data
lr_pred = lr_model.transform(test_df)

In [0]:
# Import required libraries
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Initialize binary classification evaluator
evaluator = BinaryClassificationEvaluator(
    labelCol="label"
)

# Evaluate model on predictions
lr_auc = evaluator.evaluate(lr_pred)

print("Logistic AUC:", lr_auc)

Logistic AUC: 1.0


In [0]:
# Import required libraries
from pyspark.ml.classification import RandomForestClassifier

# Initialize random forest model
rf = RandomForestClassifier(
    featuresCol="features",
    labelCol="label",
    numTrees=20,
    maxDepth=5
)

# Fit model on training data
rf_model = rf.fit(train_df)

# Make predictions on test data
rf_pred = rf_model.transform(test_df)

In [0]:
# Evaluate model on predictions
rf_auc = evaluator.evaluate(rf_pred)

print("RandomForest AUC:", rf_auc)

RandomForest AUC: 1.0


In [0]:
# Compare AUC of both models
print("Logistic:", lr_auc)
print("RandomForest:", rf_auc)

Logistic: 1.0
RandomForest: 1.0


Understanding the High AUC Score

The model produced an AUC close to 1.0, which is unusual in real-world machine learning problems and suggests that the model may be learning patterns that are too easy or directly related to the target variable.

Possible reasons include:

→ Feature Leakage
Some features may directly reveal whether a purchase happened. For example, purchase-related behavior such as cart activity or similar signals can allow the model to predict the label without truly learning user behavior.

→ Very Strong Predictive Patterns
When the separation between buyers and non-buyers is very clear, models can achieve extremely high performance.

→ Effects of Sampling and Dataset Size
The dataset was balanced using sampling to handle class imbalance. This can sometimes create a cleaner dataset where buyers and non-buyers become easier to distinguish than in real-world scenarios.